In [ ]:
#increase page width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#create a button to run all following cells 
HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_below()"><input type="submit" id="toggleButton" value="Run all cells below"></form>''')

# Modes of the new model (1D)

As shown in the notes, the computation of the modes leads to the following stability matrix
\begin{align}
  Tr(M) &= -K k^2 - \left( 1+\alpha  \frac{\overline{m}}{r} \right) \tau_{v}^{-1} - \frac{r}{\overline{m}\tau_{m}} + \overline{X}r \tau_m - D_mk^2 \label{eq:Tr_remodel-adim}\\
    det(M) &= k^4 K D_m + k^2 \left( \frac{K r}{\overline{m}\tau_m} + (1+\alpha \frac{\overline{m}}{r}) \tau_v^{-1} D_m \right)  +(1+\alpha \frac{\overline{m}}{r}) \tau_v^{-1} \left( \frac{r}{\overline{m}\tau_m}-\overline{X} r \tau_m \right)  \label{eq:Det_remodel-adim} 
\end{align}

Where we recall the definition $\overline{X}=  \chi (1-\overline{m})$
The higest higenvalue is then given by : 
\begin{equation}
\lambda_+ = \text{Tr} - \sqrt{\text{Tr}^2 - 4 \text{Det}} 
\end{equation}
Next we plot $\lambda_+(k)$, namely the dispersion relation of our 1D system.


In [ ]:
import numpy as np

from numpy import linalg as LA
import math
from numpy.lib import scimath
from precompute_fp import fp_myo
import bz2
import pickle
import os 
import sys

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi']= 150 #sets ploted image dpi (for the inline backend)
matplotlib.rcParams['savefig.dpi'] = 150 #dpi for the notebook backend

from ipywidgets import interactive,fixed,FloatSlider,HBox, Layout,VBox
import ipywidgets as widgets
# nb :
# M(Tbar,Bbar)

# feng parameters :      tbar =1.7731898882334818
#                        bbar=4.6437261959339562
#bbar = 4.64373
#tbar = 2.47669
# 1(stable upper) branch mbar = 0.92598269226757346          
#                        xbar = 0.3437161108695741
#                   
#---------------- 

class modes() :

     def __init__(self,K=1.0,tau_v=30,koff=0.14,kon=0.14,alpha=10,Dm=1.0):      
          self.Ks=np.linspace(0,1,100) #frequency range
          self.myofp = fp_myo() #load the myosin fixed points
          self.Bbars,self.Tbars,self.M0,self.M1,self.M2 = self.myofp.Bbars,self.myofp.Tbars,self.myofp.M0,self.myofp.M1,self.myofp.M2
          self.Tgrid,self.Bgrid = np.meshgrid(self.Tbars,self.Bbars,indexing='ij')
            
     def stabmat(self,k,mbar,bbar,K,tau_v,r,alpha,Dm,tau_m) : # tau_m and k0 are not relevant for the eigenvalues so we assume = 1 
         with np.errstate(divide='ignore', invalid='ignore'): #ignore divide by zero     
             T = - k**2*K* - Dm*k**2  - (1+alpha*mbar/r)/tau_v  - r/(mbar*tau_m) + bbar*(1-mbar)*r*tau_m
             D = k**4*K*Dm + k**2*((K*r/(mbar) + (1+alpha*mbar/r)*Dm/tau_v)) + (1+alpha*mbar/r)/tau_v*(r/(mbar*tau_m)-bbar*(1-mbar)*r)
             Delta = T - 4*D
             L1 =   0.5*(T+ np.lib.scimath.sqrt(Delta)) #most unstable wavelength in all case         
         return L1

     def compute_modes(self,k,m,b,K,tau_v,r,alpha,Dm,tau_m) :     
          if m == np.NaN :
               return np.NaN*np.ones(len(k))
          return self.stabmat(k,m,b,K,tau_v,r,alpha,Dm,tau_m) 
          
    
modes=modes()
k = np.linspace(0,1,100)

init = 0

def plot_modes(k,K,r,tau_v,xstar,bbar,alpha,Dm,tau_m) :
    global p0,p1,p2,fig1,ax1,lgd,init
    #compute mbar branches
    tbar = xstar*bbar
    i = np.abs(modes.Tbars-tbar).argmin()#find closest value in precomputed data
    j = np.abs(modes.Bbars-bbar).argmin()#find closest value in precomputed data
    mbars = [modes.M0[i][j],modes.M1[i][j],modes.M2[i][j]]
    lam = [modes.compute_modes(k,mbar,bbar,K,tau_v,r,alpha,Dm,tau_m).real for mbar in mbars]
    p0.set_ydata(lam[0].real)
    p1.set_ydata(lam[1].real)
    p2.set_ydata(lam[2].real)
    [lgd.get_texts()[i].set_text(r'Re($\lambda_{%d+}, m=%f$)'%(i,mbars[i])) for i in range(len(lam))]
    fig1.canvas.draw()
    fig1.canvas.flush_events()

interactive_modes = interactive(plot_modes,
                               bbar=FloatSlider(value=4.0,min=min(modes.Bbars), max=max(modes.Bbars),step=0.05, continuous_update=True),
                               K=FloatSlider(value=1.0,min=0, max=1e1, continuous_update=True),
                               tau_v=FloatSlider(value=30.0,min=0.0, max=1e2, step=1.0, continuous_update=True),
                               tau_m=FloatSlider(value=1,min=0, max=10, step=0.01, continuous_update=True),
                               r=FloatSlider(value=1.0,min=1e-2, max=1e2, step=0.01, continuous_update=True),
                               xstar=FloatSlider(value=0.5,min=min(modes.Tbars)/max(modes.Bbars),max=max(modes.Tbars)/min(modes.Bbars),step=0.01,continuous_update=True),
                               Dm=FloatSlider(value=0.0,min=0.0, max=2, step=0.01, continuous_update=True),
                               alpha=FloatSlider(value=0.0,min=0.0, max=10, step=0.01, continuous_update=True),
                               k=fixed(k)
)



#dummy plot
plt.ioff()
plt.clf()
fig1,ax1 =plt.subplots()

ax1.set_xlim(0,1)
ax1.set_ylim(-1,1)
ax1.set_xlabel(r"$k$")
ax1.set_ylabel(r"$\omega$")
p0, = ax1.plot(k**2,np.zeros_like(k),label=" ")
p1, = ax1.plot(k**2,np.zeros_like(k),label=" ")
p2, = ax1.plot(k**2,np.zeros_like(k),label=" ")
lgd = ax1.legend()

controls = VBox(interactive_modes.children[:-1], layout = Layout(flex_flow='column wrap'))
display(HBox([controls, fig1.canvas], layout = Layout(align_items='center')))
fig1.canvas.layout.min_height = '800px'
fig1.canvas.toolbar_position="top"
fig1.canvas.toolbar_visible=False 


# Nullclines (and dynamics) of a single junction
Let's now see how the nullclines are affected by the terms.
The $\lambda$ nullcine is modified whereas the $m$ one isn't  :
\begin{align}
0 &=  - \beta m -   \left( K+\tau_{v}^{-1}\left(1+\alpha m_\lambda \right) \right) \lambda + F \nonumber \\ 
0 &= 1+ m f(T) \nonumber
\end{align}
We end up with the final form : 

\begin{align}
\lambda_{\lambda}(m) &=  - \frac{\beta}{K+\tau_{v}^{-1}\left(1+\alpha m_\lambda \right)} m  + \frac{F}{K+\tau_{v}^{-1}\left(1+\alpha m_\lambda \right)}\label{eq:nullcline_lamba2} \\
\lambda_m(m) &= \frac{1}{K} \left( - \frac{\log( \frac{1}{r\,m} -1)}{k_0} - \beta m + T^{*} \right)  \label{eq:nullcline_m2}
\end{align}


In [ ]:
#capturing widget 
out = widgets.Output(layout={'border': '1px solid black'})
#@out.capture(clear_output=True) #to put just before the function definition to catch the return/errors 

In [ ]:
class nullclines_0d_remodel() :
    def __init__(self) :                                                                                         
        self.ll = lambda  m,beta,K,tau_v,r,tstar,alpha,Dm,F,k0 :  -beta*m/((K+tau_v**(-1))*(1+alpha*m)) + F/(K+tau_v**(-1)*(1+alpha*m)) 
        self.lm = lambda  m,beta,K,tau_v,r,tstar,alpha,Dm,F,k0 :  1/K* (-np.log(1/(r*m)-1)/k0 - beta*m + tstar)

        self.k0=4.0 #set k0 to an arbitrary value
        self.ms = np.linspace(0,1,100)
        debug_view = widgets.Output(layout={'border': '1px solid black'})
    
        self.interactive_nullclines = interactive(self.plot_nullclines,      
                                       beta=FloatSlider(value=1.0,min=0, max=10,step=0.01, continuous_update=True),
                                       K=FloatSlider(value=1.0,min=0, max=1e1,step=0.01, continuous_update=True),
                                       F=FloatSlider(value=1.0,min=0, max=10,step=0.01, continuous_update=True),
                                       tau_v=FloatSlider(value=30,min=0, max=1e2, step=0.01, continuous_update=True),
                                       r=FloatSlider(value=1.0,min=1e-2, max=1e2, step=0.01, continuous_update=True),
                                       tstar=FloatSlider(value=0.5,min=0, max=10,step=0.01, continuous_update=True),
                                       Dm=FloatSlider(value=0,min=0, max=2, step=0.01, continuous_update=True),
                                       alpha=FloatSlider(value=0,min=0, max=10, step=0.01, continuous_update=True),                               
                                       m=fixed(self.ms) 
                                      )

        #plt.ioff()
        #plt.clf()
                                                                                             
        self.fig2,self.ax = plt.subplots()
        self.ax.set_xlim(0,1)
        self.ax.set_ylim(-1,1)
        self.p0, = self.ax.plot(self.ms,np.zeros_like(self.ms),label=r"$\lambda$ nullcline ")
        self.p1, = self.ax.plot(self.ms,np.zeros_like(self.ms),label=r"$m$ nullcline")
        self.lgd = self.ax.legend()
        #self.interactive_nullclines
        self.controls = VBox(self.interactive_nullclines.children[:-1], layout = Layout(flex_flow='column wrap'))
        
        
        display(HBox([self.controls, self.fig2.canvas], layout = Layout(align_items='center')))
        self.fig2.canvas.layout.min_height = '800px'
        self.fig2.canvas.toolbar_position="top"
        self.fig2.canvas.toolbar_visible=False
     
    
    def plot_nullclines(self,m,beta,F,K,tau_v,r,tstar,alpha,Dm) :
        self.p0.set_ydata(self.ll(m,beta,K,tau_v,r,tstar,alpha,Dm,F,self.k0))
        self.p1.set_ydata(self.lm(m,beta,K,tau_v,r,tstar,alpha,Dm,F,self.k0))
        self.fig2.canvas.draw()
        self.fig2.canvas.flush_events()
        print(self.lm(m,beta,K,tau_v,r,tstar,alpha,Dm,F,self.k0))

nc = nullclines_0d_remodel()

                                                                                             
                                                                                            

# Dynamics in adimensional form
Since we are working with adimensional units we need to write the dynamics of the system in these units as well.
We end up with the following system :
\begin{align}
  \frac{d\overline{m}}{dt} &= \tau_m^{-1} r \left( 1 - \overline{m} \left( 1+ e^{-\chi \left(\overline{m} + \frac{K}{\beta}\lambda - x^{*}\right)} \right) \right) \\
                             \frac{d\lambda}{dt} &= -K \lambda -\frac{\beta}{r}\overline{m}-\tau_v^{-1} \left( 1+\alpha \frac{\overline{m}}{r} \right)
\end{align}

<!-- nb: we absorb $\tau_m$ by rescaling time (and therefore K,$\beta$,etc.) -->



In [ ]:
# import numpy as np

# from numpy import linalg as LA
# import math
# from numpy.lib import scimath

# import bz2
# import pickle
# import os 
# import sys

# %matplotlib widget
# import matplotlib
# import matplotlib.pyplot as plt
# matplotlib.rcParams['figure.dpi']= 200 #sets ploted image dpi (for the inline backend)
# matplotlib.rcParams['savefig.dpi'] = 200 #dpi for the notebook backend

# from ipywidgets import interactive,fixed,FloatSlider,HBox, Layout,VBox
# import ipywidgets as widgets



#Add the actual dynamics 
ll = lambda  mbar,bbar,xstar,K,tau_v,r,alpha,F :  -bbar*mbar/((K+tau_v**(-1))*(1+alpha*mbar/r)) + F/(K+tau_v**(-1)*(1+alpha*mbar/r)) 
lm = lambda  mbar,bbar,xstar,K,tau_v,r,alpha,F :  1/K* (-np.log(1/mbar-1) - bbar*mbar + xstar*bbar)


def plot_nullclines(bbar,xstar,K,tau_v,r,alpha,F) :
    global p0,p1,fig,ax,lgd,mbars  
    p0.set_ydata(ll(mbars,bbar,xstar,K,tau_v,r,alpha,F))
    p1.set_ydata(lm(mbars,bbar,xstar,K,tau_v,r,alpha,F))
    fig.canvas.draw()
    fig.canvas.flush_events()
       

#------------- integration 
from scipy.integrate import odeint

def integration_domain(dt,tmax) :
    global tdom 
    tdom = np.arange(0,tmax,dt)
    
def dmdt(lam,mb,beta,K,bbar,xstar,tau_v,r,alpha,F) :
    tb =  mb + K*r/beta*lam
    return r*(1-mb*(1+np.exp(-bbar*(tb-xstar))))

def dlamdt(lam,mb,beta,K,bbar,xstar,tau_v,r,alpha,F) :
    return -K*lam - beta*mb/r - tau_v**(-1)*(1+alpha*mb/r) + F

def sys(y,t,beta,bbar,xstar,K,tau_v,r,alpha,F):
    lam,m = y    
    return [dlamdt(lam,m,beta,bbar,xstar,K,tau_v,r,alpha,F),dmdt(lam,m,beta,bbar,xstar,K,tau_v,r,alpha,F)]
        
def simulate(b):
    global fig,ax,lgd,tdom,lamt,mt
    #need to retrieve variables 
    beta = slider_beta.value
    bbar = slider_bbar.value
    xstar = slider_xstar.value
    K = slider_K.value
    tau_v = slider_tau_v.value
    r = slider_r.value
    alpha = slider_alpha.value
    F = slider_F.value
    y0 = [slider_lam0.value,slider_m0.value]
    dt=slider_dt.value
    lamt,mt =odeint(sys, y0, tdom,args=(beta,bbar,xstar,K,tau_v,r,alpha,F)).T #transpose the result
    y = np.zeros((len(tdom),2))
    y[0] = y0
    for i,time in enumerate(tdom[1:],start=1) : 
        y[i] = y[i-1]+dt*np.array(sys(y[i-1],time,beta,bbar,xstar,K,tau_v,r,alpha,F))
    lamt,mt = y.T 
    p2.set_xdata(mt)
    p2.set_ydata(lamt)
    ax2.cla()
    ax2.plot(tdom,mt,label="m(t)")
    ax2.plot(tdom,lamt,label=r"$\lambda(t)$")
    ax2.legend(loc="upper center",prop={'size': 5})
    fig.canvas.draw()
    fig.canvas.flush_events()


#-------- sliders,buttons 


slider_beta=FloatSlider(value=1.0,min=0, max=10,step=0.01, continuous_update=False,description="beta")
slider_bbar=FloatSlider(value=4.0,min=0, max=10,step=0.01, continuous_update=False,description="bbar")
slider_xstar=FloatSlider(value=0.5,min=0, max=1,step=0.01, continuous_update=False,description="xstar")
slider_K=FloatSlider(value=1.0,min=0, max=1e1,step=0.01, continuous_update=False,description="K")
slider_tau_v=FloatSlider(value=30,min=0, max=1e2, step=0.01, continuous_update=False,description="tau_v")
slider_r=FloatSlider(value=1.0,min=1e-2, max=1e2, step=0.01, continuous_update=False,description="r")
slider_alpha=FloatSlider(value=0,min=0, max=10, step=0.01, continuous_update=False,description="alpha")                               
slider_F=FloatSlider(value=2.0,min=0, max=10, step=0.01, continuous_update=False,description="F")
slider_m0 = FloatSlider(value=0.5,min=0, max=1, step=0.01, continuous_update=False,description="m_0")
slider_lam0 = FloatSlider(value=0.0,min=0, max=1, step=0.01, continuous_update=False,description="lam_0")

interactive_nullclines = widgets.interactive_output(plot_nullclines,{ 'bbar':slider_bbar,
                                                      'xstar':slider_xstar,
                                                      'K':slider_K,
                                                      'tau_v':slider_tau_v,
                                                      'r':slider_r,
                                                      'alpha':slider_alpha, 
                                                      'F':slider_F,
                                                     }
                                    )

slider_dt = widgets.FloatLogSlider(
    value=1e-2,
    base=10,
    min=-5, # max exponent of base
    max=1, # min exponent of base
    step=1, # exponent step
    description='dt(log scale)'
)

slider_tmax = widgets.FloatLogSlider(
    value=1e2,
    base=10,
    min=2, # max exponent of base
    max=6, # min exponent of base
    step=1, # exponent step
    description='tmax(log scale)'
)


widgets.interactive_output(integration_domain,{"dt":slider_dt,"tmax":slider_tmax})

button = widgets.Button(description="Simulate ")
button.on_click(simulate)


#-------- parameters
beta=1.0
bbar=4.0
xstar=0.5
K=1.0
tau_v=30
r=1.0
alpha=0
F=2.0


#---- integration and nullclines
#y0 = [0.5,0.0]
mbars = np.linspace(0,0.99,100) #nullclines range

#-------- plot the interface,including the figure
controls_integ = VBox([button,slider_dt,slider_tmax,slider_lam0,slider_m0],layout = Layout(flex_flow='column wrap',border="solid"))
controls = VBox([controls_integ,slider_beta, slider_bbar, slider_xstar,slider_K,slider_tau_v,slider_r,slider_alpha,slider_F],layout = Layout(flex_flow='column wrap'))
#plt.ioff()
#plt.clf()
fig=plt.figure()
ax=fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax.set_xlim(0,1)
ax.set_ylim(-1,1)
ax.set_xlabel(r"$m$")
ax.set_ylabel(r"$\lambda$")

p0, = ax.plot(mbars,ll(mbars,bbar,xstar,K,tau_v,r,alpha,F),label=r"$\lambda$ nullcline ")
p1, = ax.plot(mbars,lm(mbars,bbar,xstar,K,tau_v,r,alpha,F),label=r"$m$ nullcline")
p2, = ax.plot(np.zeros_like(tdom),np.zeros_like(tdom),'r',label="simulation")
lgd = ax.legend(loc="upper center",prop={'size': 5})
p3, = ax2.plot(np.zeros_like(tdom),np.zeros_like(tdom),label="m(t)")
p4, = ax2.plot(np.zeros_like(tdom),np.zeros_like(tdom),label=r"$\lambda(t)$")

display(HBox([controls, fig.canvas], layout = Layout(align_items='center')))
# fig.canvas.layout.min_height = '800px'
fig.canvas.toolbar_position="top"
#fig.canvas.toolbar_visible=False 


# Dynamics in the original units 

In [ ]:
#the same in original units
import numpy as np
from numpy import linalg as LA
import math
from numpy.lib import scimath

import bz2
import pickle
import os 
import sys

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi']= 150 #sets ploted image dpi (for the inline backend)
matplotlib.rcParams['savefig.dpi'] = 150 #dpi for the notebook backend

from ipywidgets import interactive,fixed,FloatSlider,HBox, Layout,VBox
import ipywidgets as widgets
from scipy.integrate import odeint



#Add the actual dynamics 
ll = lambda  m,beta,Tstar,k0,K,tau_m,tau_v,r,alpha,F :  (-beta*m+F)/(K+tau_v**(-1)*(1+alpha*m)) 
lm = lambda  m,beta,Tstar,k0,K,tau_m,tau_v,r,alpha,F:  1/K* (-np.log(1/(r*m)-1)/k0 - beta*m + Tstar)


def plot_nullclines(beta,Tstar,k0,K,tau_m,tau_v,r,alpha,F) :
    global p0,p1,fig,ax,lgd,ms  
    p0.set_ydata(ll(ms,beta,Tstar,k0,K,tau_m,tau_v,r,alpha,F))
    p1.set_ydata(lm(ms,beta,Tstar,k0,K,tau_m,tau_v,r,alpha,F))
    fig.canvas.draw()
    fig.canvas.flush_events()
       

#------------- integration 

def integration_domain(dt,tmax) :
    global tdom 
    tdom = np.arange(0,tmax,dt)
    

def dydt(y,t,beta,K,k0,Tstar,tau_m,tau_v,r,alpha,F):
    lam,m = y
    T = beta*m + K*lam 
    dlamdt= -T -lam*(1+alpha*m)/tau_v + F 
    dmdt = 1/tau_m-m/tau_m*r*(1+np.exp(-k0*(T-Tstar)))
    return [dlamdt,dmdt]
  
def euler(func,y0,tdom,args):
    dt = tdom[1]
    y = np.zeros((len(tdom),2))
    y[0] = y0
    for i,time in enumerate(tdom[:-1]): 
        y[i+1] = y[i]+dt*np.array(func(y[i],time,*args))
    lamt,mt = y.T 
    return lamt,mt     

def simulate(xxxx):
    beta = slider_beta.value
    Tstar = slider_Tstar.value
    k0 = slider_k0.value
    K = slider_K.value
    tau_m = slider_tau_m.value
    tau_v = slider_tau_v.value
    r = slider_r.value
    alpha = slider_alpha.value
    F = slider_F.value
    y0 = [slider_lam0.value,slider_m0.value]
    
    lamt_scp,mt_scp = odeint(dydt, y0, tdom,args=(beta,K,k0,Tstar,tau_m,tau_v,r,alpha,F)).T #transpose the result
    p2.set_data(mt_scp,lamt_scp)
    

    ax2.clear()
    ax2_twin.clear()
    
    ax2.plot(tdom,mt_scp,'r',label='m(t)')
    ax2_twin.plot(tdom,lamt_scp,'b',label=r'$\lambda(t)$')

    ax2.legend(loc='upper center',prop={'size': 5})
    ax2_twin.legend(loc='upper right',prop={'size':5})
    ax2.set_xlabel('t')
    ax2.set_ylabel('m',color='red')
    ax2.tick_params(axis='y', colors='red')
    ax2_twin.set_ylabel(r'$\lambda$',color='blue')
    ax2_twin.tick_params(axis='y', colors='blue')
    
    fig.canvas.draw()
    fig.canvas.flush_events()


#-------- parameters beta,K,k0,Tstar,tau_m,tau_v,r,alpha,F
beta=2.3             
K=1.0                
k0=4.0               
Tstar=1.15           
tau_m=2.0            
tau_v=20             
r=1.0                
alpha=10.0           
F=1.15     

#-------- sliders,buttons 


slider_beta=FloatSlider(value=beta,min=0, max=10,step=0.01, continuous_update=False,description='beta')
slider_k0=FloatSlider(value=k0,min=0, max=10,step=0.01, continuous_update=False,description='k0')
slider_Tstar=FloatSlider(value=Tstar,min=0, max=10,step=0.01, continuous_update=False,description='Tstar')
slider_K=FloatSlider(value=K,min=0, max=1e1,step=0.01, continuous_update=False,description='K')
slider_tau_m=FloatSlider(value=tau_m,min=0, max=1e1,step=0.01, continuous_update=False,description='tau_m')
slider_tau_v=FloatSlider(value=tau_v,min=0, max=1e4, step=1.0, continuous_update=False,description='tau_v')
slider_r=FloatSlider(value=r,min=1e-2, max=1e2, step=0.01, continuous_update=False,description='r')
slider_alpha=FloatSlider(value=alpha,min=0.0, max=10, step=0.01, continuous_update=False,description='alpha')                               
slider_F=FloatSlider(value=F,min=0, max=10, step=0.01, continuous_update=False,description='F')
slider_m0 = FloatSlider(value=0.55,min=0, max=1, step=0.01, continuous_update=False,description='m_0')
slider_lam0 = FloatSlider(value=0.0,min=0, max=1, step=0.01, continuous_update=False,description='lam_0')

interactive_nullclines = widgets.interactive_output(plot_nullclines,{ 'k0':slider_k0,
                                                                     'beta':slider_beta,
                                                                     'Tstar':slider_Tstar,
                                                                     'K':slider_K,
                                                                     'tau_m':slider_tau_m,
                                                                     'tau_v':slider_tau_v,
                                                                     'r':slider_r,
                                                                     'alpha':slider_alpha, 
                                                                     'F':slider_F,
                                                                     }
                                                    )

slider_dt = widgets.FloatLogSlider(
    value=1e-2,
    base=10,
    min=-5, # max exponent of base
    max=1, # min exponent of base
    step=1, # exponent step
    description='dt(log scale)'
)

slider_tmax = widgets.FloatLogSlider(
    value=5e1,
    base=10,
    min=-1, # max exponent of base
    max=6, # min exponent of base
    step=1, # exponent step
    description='tmax(log scale)'
)


widgets.interactive_output(integration_domain,{'dt':slider_dt,'tmax':slider_tmax})

button = widgets.Button(description='Simulate')
button.on_click(simulate)

#---- integration and nullclines
ms = np.linspace(0,0.99,100) #nullclines range

#-------- plot the interface,including the figure
controls_integ = VBox([button,slider_dt,slider_tmax,slider_lam0,slider_m0],layout = Layout(flex_flow='column wrap',border='solid'))
controls = VBox([controls_integ,slider_beta, slider_K, slider_k0,slider_Tstar,slider_tau_m,slider_tau_v,slider_r,slider_alpha,slider_F],layout = Layout(flex_flow='column wrap'))
plt.ioff()
plt.clf()
fig=plt.figure()
ax=fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax2_twin = ax2.twinx()


ax.set_xlim(0,1)
ax.set_ylim(-1,1)
ax.set_xlabel(r'$m$')
ax.set_ylabel(r'$\lambda$')

p0, = ax.plot(ms,ll(ms,beta,Tstar,k0,K,tau_m,tau_v,r,alpha,F),label=r'$\lambda$ nullcline')
p1, = ax.plot(ms,lm(ms,beta,Tstar,k0,K,tau_m,tau_v,r,alpha,F),label=r'$m$ nullcline')
p2, = ax.plot(np.zeros_like(tdom),np.zeros_like(tdom),'r',label='simulation')
lgd = ax.legend(loc='upper center',prop={'size': 5})
p3, = ax2.plot(np.zeros_like(tdom),np.zeros_like(tdom),label='m(t)')
p4, = ax2.plot(np.zeros_like(tdom),np.zeros_like(tdom),label=r'$\lambda(t)$')

display(HBox([controls, fig.canvas], layout = Layout(align_items='center')))
fig.canvas.layout.min_height = '1000px'
fig.canvas.toolbar_position='top'
#fig.canvas.toolbar_visible=False 
